In [35]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [36]:
dir_path = rf'D:\OneDrive\Documentos\Materiais didáticos\MECAI\Dissertação\script\Results'
name_path = 'results.csv'
complete_path = os.path.join(dir_path, name_path)

df_res = pd.read_csv(complete_path, encoding='latin-1')
df_res = df_res.loc[df_res['Valor'] != 0, :]
df_res.loc[:, 'Tempo'] = df_res.loc[:, 'Tempo'] + 1

In [37]:
dir = rf'D:\OneDrive\Documentos\Materiais didáticos\MECAI\Dissertação\script\Dados Gerados Mestrado\Auxiliar'

# df_cluster_cds_cities = pd.read_csv(os.path.join(dir, 'df_cluster_cds_cities.csv'))
# df_cluster_pa_cities = pd.read_csv(os.path.join(dir, 'df_cluster_pa_cities.csv'))
df_dist_CD_PA = pd.read_csv(os.path.join(dir, 'df_dist_CD_PA.csv'))
df_dist_ZD_PA = pd.read_csv(os.path.join(dir, 'df_dist_ZD_PA.csv'))
df_instalacao = pd.read_csv(os.path.join(dir, 'df_instalacao.csv'))
df_desinstalacao = pd.read_csv(os.path.join(dir, 'df_desinstalacao.csv'))
df_dist_Fab_CD = pd.read_csv(os.path.join(dir, 'df_dist_Fab_CD.csv'))
df_dist_CD_Desc = pd.read_csv(os.path.join(dir, 'df_dist_CD_Desc.csv'))
# df_regiao_pa_zd = pd.read_csv(os.path.join(dir, 'df_regiao_pa_zd.csv'))

df_estq_inicial = pd.read_csv(os.path.join(dir, 'df_estq_inicial.csv'))
df_estq_inicial = df_estq_inicial.loc[:, ['PA', 'lim_estq_in_pa', 'Restr_Rec', 'Restr_Env']]

In [38]:
def pivotar_demanda_mensal(df, colunas_valor, name_col, name_col_pivot, mapa=None):
    """
    Função para pivotar o DataFrame de instalação, transformando as colunas de demanda mensal
    em uma coluna única com os meses representados por números de 0 a 11.
    
    Parâmetros:
    df_instalacao (DataFrame): DataFrame contendo as colunas de demanda mensal.
    
    Retorna:
    DataFrame: DataFrame pivotado com colunas 'ZD', 'mes' e 'demanda'.
    """
 
    # Aplicando melt
    df = df[[name_col] + colunas_valor].melt(
        id_vars=name_col,
        value_vars=colunas_valor,
        var_name= name_col_pivot,
        value_name='demanda'
    )

    if mapa is not None:
        df[name_col_pivot] = df[name_col_pivot].replace(mapa)

    return df

# Lista com os nomes originais das colunas de demanda mensal
colunas_demanda = [
    'demanda_normal_Jan', 'demanda_normal_Fev', 'demanda_normal_Mar',
    'demanda_normal_Abr', 'demanda_normal_Mai', 'demanda_normal_Jun',
    'demanda_normal_Jul', 'demanda_normal_Ago', 'demanda_normal_Set',
    'demanda_normal_Out', 'demanda_normal_Nov', 'demanda_normal_Dez'
]

# Dicionário para mapear os meses para valores de 1 a 12
mapa_meses = {
    'demanda_normal_Jan': 1, 'demanda_normal_Fev': 2, 'demanda_normal_Mar': 3,
    'demanda_normal_Abr': 4, 'demanda_normal_Mai': 5, 'demanda_normal_Jun': 6,
    'demanda_normal_Jul': 7, 'demanda_normal_Ago': 8, 'demanda_normal_Set': 9,
    'demanda_normal_Out': 10, 'demanda_normal_Nov': 11, 'demanda_normal_Dez': 12
}

# Substituir nomes dos meses pelos índices numéricos
df_inst_pivot = pivotar_demanda_mensal(df_instalacao, colunas_demanda, 'ZD', 'mes', mapa_meses)
df_dinst_pivot = pivotar_demanda_mensal(df_desinstalacao, colunas_demanda, 'ZD', 'mes', mapa_meses)

C:\Users\rpafe\AppData\Local\Temp\ipykernel_20136\2741838635.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[name_col_pivot] = df[name_col_pivot].replace(mapa)
C:\Users\rpafe\AppData\Local\Temp\ipykernel_20136\2741838635.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[name_col_pivot] = df[name_col_pivot].replace(mapa)


In [39]:
import pandas as pd

def union_dfs_with_different_columns(list_dfs):
    """
    Realiza a união (concatenação) de quatro DataFrames, cada um com 3 colunas,
    mesmo que os nomes das colunas sejam diferentes.

    Args:
        df1 (pd.DataFrame): O primeiro DataFrame.
        df2 (pd.DataFrame): O segundo DataFrame.
        df3 (pd.DataFrame): O terceiro DataFrame.
        df4 (pd.DataFrame): O quarto DataFrame.

    Returns:
        pd.DataFrame: Um novo DataFrame resultante da união dos quatro DataFrames,
                      com as colunas renomeadas para 'coluna_1', 'coluna_2', 'coluna_3'.
    """
    dfs = [list_dfs[i] for i in range(len(list_dfs))]
    unified_dfs = []

    for i, df in enumerate(dfs):
        if df.shape[1] != 3:
            raise ValueError(f"DataFrame {i+1} não possui 3 colunas. Possui {df.shape[1]}.")

        # Renomeia as colunas para um padrão comum
        df_renamed = df.copy()
        df_renamed.columns = ['Origem_Dist', 'Destino_Dist', 'Distancia']
        unified_dfs.append(df_renamed)

    # Concatena todos os DataFrames renomeados
    union_df = pd.concat(unified_dfs, ignore_index=True)

    return union_df

In [40]:
colunas_cd_pas = [each for each in df_dist_CD_PA.columns if each[:3] == 'PA_']
df_dist_CD_PA_pivot = pivotar_demanda_mensal(df_dist_CD_PA, colunas_cd_pas, 'CD', 'PA')

colunas_pas_zd = [each for each in df_dist_ZD_PA.columns if each[:3] == 'PA_']
df_dist_ZD_PA_pivot = pivotar_demanda_mensal(df_dist_ZD_PA, colunas_pas_zd, 'ZD', 'PA')

colunas_cd_fab = [each for each in df_dist_Fab_CD.columns if each[:3] == 'CD_']
df_dist_Fab_CD_pivot = pivotar_demanda_mensal(df_dist_Fab_CD, colunas_cd_fab, 'FAB', 'CD')

colunas_cd_desc = [each for each in df_dist_CD_Desc.columns if each[:3] == 'CD_']
df_dist_CD_Desc_pivot = pivotar_demanda_mensal(df_dist_CD_Desc, colunas_cd_desc, 'DESC', 'CD')

list_dfs = [df_dist_CD_PA_pivot, df_dist_ZD_PA_pivot, df_dist_Fab_CD_pivot, df_dist_CD_Desc_pivot]
df_dist = union_dfs_with_different_columns(list_dfs)

In [41]:
def left_join(df1, df2, left_col, right_col, prefix, how='left'):
    # Descobrir colunas que não são chaves
    non_key_cols = [col for col in df2.columns]
        
    # Fazer o merge normalmente
    df = df1.merge(df2, left_on=left_col, right_on=right_col, how=how)

    # Renomear colunas do df2 com prefixo
    df = df.copy().rename(columns={col: f"{prefix}{col}" for col in non_key_cols})
    
    # Remover a coluna de chave do df2
    right_list = [f"{prefix}{right_col[i]}" for i in range(len(right_col)) if f"{prefix}{right_col[i]}" in df.columns]
    df = df.drop(columns=right_list, errors='ignore')

    return df

In [42]:
df_res = left_join(df_res, df_inst_pivot, ['Destino', 'Tempo'], ['ZD', 'mes'], 'p_inst_')
df_res = left_join(df_res, df_dinst_pivot, ['Destino', 'Tempo'], ['ZD', 'mes'], 'p_dinst_')

In [43]:
df_res = left_join(df_res, df_dist, ['Origem', 'Destino'], ['Origem_Dist', 'Destino_Dist'], 'p_')
df_res = left_join(df_res, df_dist, ['Origem', 'Destino'], ['Destino_Dist', 'Origem_Dist'], 'p_aux_')
df_res['p_Distancia'] = df_res['p_Distancia'].fillna(df_res['p_aux_Distancia'])
df_res = df_res.drop(columns=['p_aux_Distancia'])

In [44]:
df_res = left_join(df_res, df_estq_inicial, ['Origem'], ['PA'], 'p_ori_pa_')
df_res = left_join(df_res, df_estq_inicial, ['Destino'], ['PA'], 'p_dest_pa_')

In [45]:
def substituir_colunas(df, list_col1, list_col2):
    for col1, col2 in zip(list_col1, list_col2):
        if col1 in df.columns and col2 in df.columns:
            df[col1] = df[col1].fillna(df[col2])
            df = df.drop(columns=[col2])
    return df

df_res = substituir_colunas(df_res, ['p_ori_pa_lim_estq_in_pa', 'p_ori_pa_Restr_Rec', 'p_ori_pa_Restr_Env'], \
        ['p_dest_pa_lim_estq_in_pa', 'p_dest_pa_Restr_Rec', 'p_dest_pa_Restr_Env'])

In [46]:
df_feat = pd.read_csv(os.path.join(dir_path, 'df_features.csv'), encoding='latin-1')
df_features = pd.read_csv(os.path.join(dir_path, 'df_features.csv'), encoding='latin-1')

df_res.loc[(df_res['Abrv Var']=='sf'), 
           'f_ori_fab_lim_estq'] = df_feat['p_ori_fab_lim_estq'].values[0]
df_res.loc[(df_res['Abrv Var'].isin(['qac'])),
           'f_fab_rest_env'] = df_feat['p_fab_rest_env'].values[0]
df_res.loc[(df_res['Abrv Var'].isin(['qrf'])),
           'f_fab_rest_rec'] = df_feat['p_fab_rest_rec'].values[0]

df_res.loc[(df_res['Abrv Var'].isin(['sc'])),
           'c_ori_cd_lim_estq'] = df_feat['p_ori_cd_lim_estq'].values[0]
df_res.loc[(df_res['Abrv Var'].isin(['qrf', 'qap', 'qrd'])),
           'c_cd_rest_env'] = df_feat['p_cd_rest_env'].values[0]
df_res.loc[(df_res['Abrv Var'].isin(['qrc', 'qac'])),
           'c_cd_rest_rec'] = df_feat['p_cd_rest_rec'].values[0]

# df_res.fillna(0, inplace=True)

In [47]:
# Seu código
def definir_origem_dest(df_res, ltr, ponto):
    list_orig_temp = []
    list_dest_temp = []
    analise_fabrica = []
    for ori, dest in zip(df_res['Origem'], df_res['Destino']):
        if isinstance(ori, str) and ori[0] == ltr:
            list_orig_temp.append(ori)
            list_dest_temp.append(dest)
        elif isinstance(dest, str) and dest[0] == ltr:
            list_orig_temp.append(dest)
            list_dest_temp.append(ori)
        else:
            list_orig_temp.append(np.nan)
            list_dest_temp.append(np.nan)
        analise_fabrica.append(ponto)
    return [list_orig_temp, list_dest_temp, analise_fabrica]

df_res.loc[:, ['Origem_Fabrica', 'Destino_Fabrica', 'Analise_Fabrica']] = list(zip(*definir_origem_dest(df_res, 'F', 'Fabrica')))
df_res.loc[:, ['Origem_CD', 'Destino_CD', 'Analise_CD']] = list(zip(*definir_origem_dest(df_res, 'C', 'CD')))
df_res.loc[:, ['Origem_PA', 'Destino_PA', 'Analise_PA']] = list(zip(*definir_origem_dest(df_res, 'P', 'PA')))

In [48]:
# Crie uma variável para a condição de filtro, evitando repetição
filtro_origem = (~df_res['Origem_Fabrica'].isna()) & (df_res['Origem_Fabrica'].str.startswith('F'))

# 1. Calcular percentual de restrição de envio
df_res.loc[filtro_origem, 'perc_restr_env_fab'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qac'])), 'Valor'] /
    df_res.loc[filtro_origem, 'f_fab_rest_env']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qac'])), 'agg_fab'] = 'Env_Fab'

# 2. Corrigir e calcular percentual de restrição de recebimento
# A lista de 'qap' estava redundante e foi simplificada para ['qap']
df_res.loc[filtro_origem, 'perc_restr_rec_fab'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qrf'])), 'Valor'] /
    df_res.loc[filtro_origem, 'f_fab_rest_rec']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qrf'])), 'agg_fab'] = 'Rec_Fab'

# 3. Calcular percentual de restrição de estoque
df_res.loc[filtro_origem, 'perc_restr_estq_fab'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sf'])), 'Valor'] /
    df_res.loc[filtro_origem, 'f_ori_fab_lim_estq']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sf'])), 'agg_fab'] = 'Estq_Fab'

del filtro_origem

In [49]:
# Crie uma variável para a condição de filtro, evitando repetição
filtro_origem = (~df_res['Origem_CD'].isna()) & (df_res['Origem_CD'].str.startswith('C'))

# 1. Calcular percentual de restrição de envio
df_res.loc[filtro_origem, 'perc_restr_env_cd'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qap', 'qrd', 'qrf'])), 'Valor'] /
    df_res.loc[filtro_origem, 'c_cd_rest_env']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qap', 'qrd', 'qrf'])), 'agg_cd'] = 'Env_CD'

# 2. Corrigir e calcular percentual de restrição de recebimento
# A lista de 'qap' estava redundante e foi simplificada para ['qap']
df_res.loc[filtro_origem, 'perc_restr_rec_cd'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qrc', 'qac'])), 'Valor'] /
    df_res.loc[filtro_origem, 'c_cd_rest_rec']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qrc', 'qac'])), 'agg_cd'] = 'Rec_CD'

# 3. Calcular percentual de restrição de estoque
df_res.loc[filtro_origem, 'perc_restr_estq_cd'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sc'])), 'Valor'] /
    df_res.loc[filtro_origem, 'c_ori_cd_lim_estq']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sc'])), 'agg_cd'] = 'Estq_CD'

del filtro_origem

In [50]:
# Crie uma variável para a condição de filtro, evitando repetição
filtro_origem = (~df_res['Origem_PA'].isna()) & (df_res['Origem_PA'].str.startswith('P'))

# 1. Calcular percentual de restrição de envio
df_res.loc[filtro_origem, 'perc_restr_env_pa'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qaj', 'qrc'])), 'Valor'] /
    df_res.loc[filtro_origem, 'p_ori_pa_Restr_Env']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qaj', 'qrc'])), 'agg_pa'] = 'Env_PA'


# 2. Corrigir e calcular percentual de restrição de recebimento
# A lista de 'qap' estava redundante e foi simplificada para ['qap']
df_res.loc[filtro_origem, 'perc_restr_rec_pa'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qap', 'qrp'])), 'Valor'] /
    df_res.loc[filtro_origem, 'p_ori_pa_Restr_Rec']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['qap', 'qrp'])), 'agg_pa'] = 'Rec_PA'

# 3. Calcular percentual de restrição de estoque
df_res.loc[filtro_origem, 'perc_restr_estq_pa'] = (
    df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sp'])), 'Valor'] /
    df_res.loc[filtro_origem, 'p_ori_pa_lim_estq_in_pa']
)
df_res.loc[filtro_origem & (df_res['Abrv Var'].isin(['sp'])), 'agg_pa'] = 'Estq_PA'

del filtro_origem

In [51]:
df_res.to_csv(os.path.join(dir_path, "df_res.csv"), encoding='latin-1', index=False)
df_res.to_excel(os.path.join(dir_path, "df_res.xlsx"), index=False)